In [ ]:
import pandas as pd

In [ ]:
#Cargamos las bases de datos

spam = pd.read_csv("http://turing.iimas.unam.mx/~gibranfp/cursos/aprendizaje_automatizado/data/spam.csv", header=None)
tumores= pd.read_csv("http://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data", header=None)

In [ ]:
# Separar cada número en una columna
df_split = spam[0].str.split(expand=True)

# Renombrar la última columna como 'target'
df_split.columns = [f'col_{i}' for i in range(df_split.shape[1]-1)] + ['target']

# Convertir a tipo numérico
df_split = df_split.apply(pd.to_numeric)

In [ ]:
y = df_split['target']           # Variable dependiente
X = df_split.drop(columns=['target'])  # Variables independientes

In [ ]:

from scipy.optimize import minimize

class LogisticRegressionML:
    def fit(self, X, y):
        """
        Estima los parámetros (pesos) por máxima verosimilitud
        """
        self.clases = np.unique(y)
        if len(self.clases) != 2:
            raise ValueError("Este modelo solo soporta clasificación binaria.")
        
        n = X.shape[0]
        self.n_atr = X.shape[1]

        # Insertamos el bias (columna de unos)
        X = np.insert(X, 0, 1, axis=1)
        self.X_train = X
        self.y_train = y
        w_init = np.zeros(X.shape[1])

        def sigmoid(z):
            return 1 / (1 + np.exp(-z))

        def loss(w):
            z = X @ w
            h = sigmoid(z)
            epsilon = 1e-5
            return -np.mean(y * np.log(h + epsilon) + (1 - y) * np.log(1 - h + epsilon))

        def gradient(w):
            z = X @ w
            h = sigmoid(z)
            return X.T @ (h - y) / n

        res = minimize(fun=loss, x0=w_init, jac=gradient, method='BFGS')
        self.w = res.x

    def predict_proba(self, X):
        """
        Calcula la probabilidad de la clase positiva (label 1)
        """
        X = np.insert(X, 0, 1, axis=1)
        z = X @ self.w
        probs = 1 / (1 + np.exp(-z))
        return np.column_stack([1 - probs, probs])  # clase 0 y clase 1

    def predict(self, X):
        """
        Predice la clase (0 o 1) según el umbral 0.5
        """
        probs = self.predict_proba(X)
        return (probs[:, 1] >= 0.5).astype(int)


In [ ]:
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
import numpy as np

# Definimos el clasificador
pipeline = make_pipeline(StandardScaler(), LogisticRegressionML())

#Creamos el objeto de validación cruzada
cv = RepeatedStratifiedKFold(n_splits=5, n_repeats=10, random_state=42)

#Calculamos los scores
scores = cross_val_score(pipeline, X, y, cv=cv, scoring='accuracy')

# Mostramos los resultados
print(f"Puntaje promedio: {scores.mean():.4f}")
print(f"Desviación estándar: {scores.std():.4f}")
